<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Notebook_PySpark_CalidadAireBogota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Datos de Calidad del Aire en Bogotá con PySpark

Este cuaderno tiene como objetivo facilitar la comprensión de los modelos de clasificación y regresión en entornos de Big Data utilizando Apache Spark. Se trabajará con un conjunto de datos público sobre calidad del aire en Bogotá, Colombia.

Los objetivos específicos son:
- Cargar y explorar datos ambientales de Bogotá.
- Aplicar regresión lineal para predecir valores continuos de concentración de contaminantes.
- Aplicar clasificación para categorizar niveles de calidad del aire.
- Demostrar el uso de `SparkSession`, `DataFrames`, y `MLlib` para construir pipelines analíticos escalables.

Este notebook puede ejecutarse en Google Colab con PySpark configurado o en plataformas como Databricks.

In [1]:
# Instalación de PySpark en Google Colab (solo ejecutar en Colab)
!pip install pyspark

In [2]:
# Importar librerías principales
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator

In [3]:
# Crear sesión de Spark
spark = SparkSession.builder.appName("CalidadAireBogota").getOrCreate()

## Cargar los datos
Puede utilizar un archivo CSV con datos de calidad del aire, por ejemplo desde [datos.gov.co](https://www.datos.gov.co/).
Para este ejercicio, simulemos un DataFrame similar con variables de interés.

### Enlace a datos reales
Puede descargar datos reales de calidad del aire en Bogotá desde el portal oficial de datos abiertos:
- [Datos históricos de calidad del aire - Bogotá](https://www.datos.gov.co/Medio-Ambiente/Calidad-del-aire-en-Bogot-D-C-historico/j8gt-88jc)

Para este notebook, si desea utilizar ese dataset, debe cargarlo desde su computador o desde una URL pública usando `pandas` y convertirlo luego a un DataFrame de PySpark.

In [5]:
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
data = [
    (30.5, 22.0, 40.3, 'Buena'),
    (55.1, 28.3, 60.2, 'Moderada'),
    (70.2, 35.1, 80.5, 'Dañina'),
    (25.3, 18.4, 35.0, 'Buena')
]
schema = StructType([
    StructField("PM10", DoubleType(), True),
    StructField("NO2", DoubleType(), True),
    StructField("O3", DoubleType(), True),
    StructField("Categoria", StringType(), True)
])
df = spark.createDataFrame(data, schema)
df.show()

+----+----+----+---------+
|PM10| NO2|  O3|Categoria|
+----+----+----+---------+
|30.5|22.0|40.3|    Buena|
|55.1|28.3|60.2| Moderada|
|70.2|35.1|80.5|   Dañina|
|25.3|18.4|35.0|    Buena|
+----+----+----+---------+



## Modelo de Regresión: predecir O3 con base en PM10 y NO2

In [7]:
features = VectorAssembler(inputCols=["PM10", "NO2"], outputCol="features")
lr = LinearRegression(featuresCol="features", labelCol="O3")
pipeline = Pipeline(stages=[features, lr])
model = pipeline.fit(df)
predictions = model.transform(df)
predictions.select("features", "O3", "prediction").show()

+-----------+----+------------------+
|   features|  O3|        prediction|
+-----------+----+------------------+
|[30.5,22.0]|40.3| 41.48194371304527|
|[55.1,28.3]|60.2|61.863393974332745|
|[70.2,35.1]|80.5|  79.2591258854577|
|[25.3,18.4]|35.0|  33.3955364271643|
+-----------+----+------------------+



### Interpretación de la salida de la regresión
La tabla mostrada por `predictions.select("features", "O3", "prediction").show()` contiene tres columnas:
- **features**: el vector de entrada compuesto por los valores de PM10 y NO2.
- **O3**: el valor real de la concentración de ozono troposférico.
- **prediction**: el valor estimado por el modelo de regresión lineal.

Al comparar las columnas `O3` y `prediction`, se puede evaluar cuán cercano está el modelo de los valores reales. Para mejorar la precisión, se pueden ajustar los hiperparámetros, aplicar transformaciones o incluir nuevas variables predictoras.

## Modelo de Clasificación: predecir la categoría de calidad del aire

In [ ]:
indexer = StringIndexer(inputCol="Categoria", outputCol="label")
lr_class = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)
pipeline_class = Pipeline(stages=[features, indexer, lr_class])
model_class = pipeline_class.fit(df)
predictions_class = model_class.transform(df)
predictions_class.select("features", "Categoria", "prediction").show()

## Reflexión final
Este ejercicio ha permitido implementar modelos analíticos sobre un conjunto de datos simulado en PySpark. El estudiante puede replicar esta experiencia utilizando un conjunto de datos real, como los disponibles en la plataforma [datos.gov.co](https://www.datos.gov.co/), para vincular el análisis con situaciones reales del contexto colombiano.

Se recomienda repetir el flujo de trabajo modificando los parámetros del modelo, aplicando nuevas transformaciones y evaluando con diferentes métricas para comprender mejor el impacto de cada ajuste en el desempeño del modelo.

### Interpretación de la salida de la clasificación
La salida `predictions_class.select("features", "Categoria", "prediction").show()` muestra:
- **features**: vector de entrada compuesto por PM10 y NO2.
- **Categoria**: la clase original de la calidad del aire.
- **prediction**: la categoría predicha por el modelo (como índice numérico).

El modelo de clasificación convierte primero las categorías en números (por ejemplo, Buena = 0.0, Moderada = 1.0, Dañina = 2.0). Luego, entrena un modelo de regresión logística multinomial para predecir la clase más probable. Esta técnica es útil para automatizar alertas de calidad del aire o alimentar tableros de control en tiempo real.